In [ ]:
# Dependencies
import requests
import json
import quandl #https://docs.quandl.com/docs/python-time-series give a how to guide
from config import q_api_key, y_api_key
import pandas as pd
import matplotlib.pyplot as plt
from uszipcode import SearchEngine, SimpleZipcode, Zipcode #https://uszipcode.readthedocs.io/index.html
#create a variable for the searchengine function which pulls down all lat/longs in database for crime/airbnb data (lat/long to zip)
search = SearchEngine()

In [ ]:
#use this call to set the api_key for zillow data
quandl.ApiConfig.api_key = q_api_key

In [ ]:
#pull in zipcode list
zipcode_df=pd.read_csv("zipcode2.csv")
#turn zipcode into a list
zipcode=zipcode_df['Zip Code'].tolist()

#ZILLOW DATA
#create a blank list for median home values and median rent from Zillow
medianrent=[]
medianhomevalues=[]
#pull median home value for each zip code in Chicago at the yearly level for 2018 only
for x in range(0,len(zipcode)):
    try:
        mr_raw=quandl.get(f"ZILLOW/Z{zipcode[x]}_ZRIAH", collapse="annual", start_date="2018-12-31", end_date="2018-12-31")
        mr_list=mr_raw["Value"].tolist()
        medianrent.append(mr_list[0])
        print(f'Pulled rent data for {zipcode[x]}')
    except:
        mr_missing='NaN'
        medianrent.append(mr_missing)
        print(f'Rent data for {zipcode[x]} not found')
    try:
        mhv_raw=quandl.get(f"ZILLOW/Z{zipcode[x]}_ZHVIAH", collapse="annual", start_date="2018-12-31", end_date="2018-12-31")
        mhv_list=mhv_raw["Value"].tolist()
        medianhomevalues.append(mhv_list[0])
        print(f'Pulled home value data for {zipcode[x]}')
    except:
        mhv_missing='NaN'
        medianhomevalues.append(mhv_missing)
        print(f'Home value data for {zipcode[x]} not found')
print(medianrent)
print(medianhomevalues)

In [ ]:
#create a dataframe of zipcode, median rent, and median home values
housing_df = pd.DataFrame({"Zipcode":zipcode, "Median Rent":medianrent, "Median Home Value":medianhomevalues})
housing_df.head()

In [ ]:
#YELP DATA
#found in https://python.gotrained.com/yelp-fusion-api-tutorial/
headers = {'Authorization': 'Bearer %s' % y_api_key}

#category of business you want to pull 1. nightlife
categories = 'nightlife' 
#base url
url='https://api.yelp.com/v3/businesses/search'
#create a list for the number of returned categories in zipcode
category_l = []
#for every zipcode in the list return all businesses for the category searched and get rid of entries not within the zip and add to a total list
for zipcode in zipcodes:
    print(f'starting datapull for {zipcode}')
    #create a list for all the pages returned
    yelp_data = []
    #the max returned businesses is 1000 and you can have 50 on a page so create a for loop to iterate through until get 400 code
    #https://stackoverflow.com/questions/35525994/how-to-request-more-than-20-results-from-yelp-api
    for offset in range(0, 1000, 50):
        params = {
            'limit': 50, 
            'location': zipcode,
            'categories': categories,
            'offset': offset
        }

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            #combine page with previous pages
            yelp_data += response.json()['businesses']
        elif response.status_code == 400:
            print('400 Bad Request')
            break
    print(f'finished datapull for {zipcode}')
    #create a blank zipcode list in preparation of pulling in zipcodes for each entry collected in the above yelp_data
    zipcode_b =[]
    #for each business in the yelp_data json, find the zip code and then append it to the zipcode_b list
    for business in range(0,len(yelp_data)):
        currentzip =str(yelp_data[business]["location"]["zip_code"])
        if currentzip == str(zipcode):
            zipcode_b.append(currentzip)
            #print(currentzip)
       # else:
        #    print(f'Not correct zip :{currentzip} != {zipcode}')
    #add to the category list with total number of categories from current zipcode you are looking at
    numberinzip=len(zipcode_b)
    category_l.append(numberinzip)
    print(f'new appended category total: {numberinzip}  for {zipcode}')

In [ ]:
#category of business you want to pull 2. restaurants
categories = 'restaurants' 
#base url
url='https://api.yelp.com/v3/businesses/search'
#create a list for the number of returned categories in zipcode
category_r = []
#for every zipcode in the list return all businesses for the category searched and get rid of entries not within the zip and add to a total list
for zipcode in zipcodes:
    print(f'starting datapull for {zipcode}')
    #create a list for all the pages returned
    yelp_data = []
    #the max returned businesses is 1000 and you can have 50 on a page so create a for loop to iterate through until get 400 code
    #https://stackoverflow.com/questions/35525994/how-to-request-more-than-20-results-from-yelp-api
    for offset in range(0, 1000, 50):
        params = {
            'limit': 50, 
            'location': zipcode,
            'categories': categories,
            'offset': offset
        }

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            #combine page with previous pages
            yelp_data += response.json()['businesses']
        elif response.status_code == 400:
            print('400 Bad Request')
            break
    print(f'finished datapull for {zipcode}')
    #create a blank zipcode list in preparation of pulling in zipcodes for each entry collected in the above yelp_data
    zipcode_b =[]
    #for each business in the yelp_data json, find the zip code and then append it to the zipcode_b list
    for business in range(0,len(yelp_data)):
        currentzip =str(yelp_data[business]["location"]["zip_code"])
        if currentzip == str(zipcode):
            zipcode_b.append(currentzip)
            #print(currentzip)
       # else:
        #    print(f'Not correct zip :{currentzip} != {zipcode}')
    #add to the category list with total number of categories from current zipcode you are looking at
    numberinzip=len(zipcode_b)
    category_r.append(numberinzip)
    print(f'new appended category total: {numberinzip}  for {zipcode}')

In [ ]:
#category of business you want to pull 3. grocery
categories = 'grocery' 
#base url
url='https://api.yelp.com/v3/businesses/search'
#create a list for the number of returned categories in zipcode
category_g = []
#for every zipcode in the list return all businesses for the category searched and get rid of entries not within the zip and add to a total list
for zipcode in zipcodes:
    print(f'starting datapull for {zipcode}')
    #create a list for all the pages returned
    yelp_data = []
    #the max returned businesses is 1000 and you can have 50 on a page so create a for loop to iterate through until get 400 code
    #https://stackoverflow.com/questions/35525994/how-to-request-more-than-20-results-from-yelp-api
    for offset in range(0, 1000, 50):
        params = {
            'limit': 50, 
            'location': zipcode,
            'categories': categories,
            'offset': offset
        }

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            #combine page with previous pages
            yelp_data += response.json()['businesses']
        elif response.status_code == 400:
            print('400 Bad Request')
            break
    print(f'finished datapull for {zipcode}')
    #create a blank zipcode list in preparation of pulling in zipcodes for each entry collected in the above yelp_data
    zipcode_b =[]
    #for each business in the yelp_data json, find the zip code and then append it to the zipcode_b list
    for business in range(0,len(yelp_data)):
        currentzip =str(yelp_data[business]["location"]["zip_code"])
        if currentzip == str(zipcode):
            zipcode_b.append(currentzip)
            #print(currentzip)
       # else:
        #    print(f'Not correct zip :{currentzip} != {zipcode}')
    #add to the category list with total number of categories from current zipcode you are looking at
    numberinzip=len(zipcode_b)
    category_g.append(numberinzip)
    print(f'new appended category total: {numberinzip}  for {zipcode}')

In [ ]:
#combine nightlife, restaurant and grocery with zipcode into dataframe
yelp_complete=pd.DataFrame({"Zipcode":zipcodes, "Total Nightlife": category_l, "Total Restaurant": category_r, 
                      "Total Grocery": category_g})

In [ ]:
#export data to csv
housing_df.to_csv("housing_data.csv", encoding="utf-8", index=False)
yelp_complete.to_csv("yelp_data.csv", encoding="utf-8", index=False)

In [ ]:
#CRIME DATA
#read in crime data
df_crime = pd.read_csv('Crimes 2018.csv')
df_crime_latlon=df_crime.loc[:,['Latitude', 'Longitude']]
df_crime_latlon

In [ ]:
res = []
#for every row in df_crime_latlon
for index, row in df_crime_latlon.iterrows():
    #part of uszipcode library it takes the lat/lon from the row you are in and looks for the nearest zip (returns=1) 
    #in a 3 mile radius
    result = search.by_coordinates(row["Latitude"], row["Longitude"], radius=3, returns=1)
    #append to res
    res.append(result)

In [ ]:
#double check that the length is the length in the csv file
len(res)

In [ ]:
#create a blank list for zipcodes
zipcodes= []
#for every entry returned in res
for i in range(len(res)):
    #this is a list of lists so for every entry there is another list inside that contains geographical information
    for x in res[i]:
        # zipcode is from uszipcode library and find the zipcode in the list
        zip = x.zipcode
     #   first = res[i]
        #append the zipcode to the zipcodes list
        zipcodes.append(zip)
#     zipcodes.append(first)
len(zipcodes)

In [ ]:
#create a dataframe of zipcodes
zip_df = pd.DataFrame({"Zipcode": zipcodes})
#count number of zip codes and reset index so zip code becomes a column
zipcode_count = zip_df["Zipcode"].value_counts().reset_index()
#rename the variables and output to csv
zipcode_count_rename = zipcode_count.rename(columns={"index": "Zipcode","Zipcode":"Crime Count"})
zipcode_count_rename.to_csv("Crime_Count by zip.csv", encoding="utf-8", index=False)
#make sure zipcode is numeric
zipcode_count_rename['Zipcode'] = pd.to_numeric(zipcode_count_rename['Zipcode'])
#output to csv
zipcode_count_rename.to_csv("crime count by zip.csv", encoding="utf-8", index=False)

In [ ]:
#plot rent vs home value to analyze which variable to use as y variable in regression/comp with other variables
fig, ax1 = plt.subplots()
ax1.set_facecolor("lavender")
plt.scatter(medianrent,medianhomevalues, edgecolor="black", c="blue", marker='o')
plt.grid(color="white")
plt.title("Median Rent vs. Median Home Values by Zipcode")
plt.ylabel("Median Home Values")
plt.xlabel("Median Rent")
plt.savefig('Median Rent vs Median Home Value.png', bbox_inches='tight')
plt.show()